# Element Parameter Detection

## Setup

In [ ]:
import numpy as np
import math
import random
import os
import os.path
import torch
import sys
import copy
import pickle
import importlib
import torch.nn as nn
import time
import functorch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
#from chamferdist import ChamferDistance
from pathlib import Path

import ifcopenshell
import open3d as o3d

from src.elements import *
from src.ifc import *
from src. preparation import *
from src.dataset import *
from src.pointnet import *
from src.visualisation import *
from src.geometry import sq_distance
from src.icp import icp_finetuning
from src.chamfer import *
from src.utils import *
from src.plots import plot_error_graph, plot_parameter_errors
from src.pca import testset_PCA
from src.finetune import chamfer_fine_tune
from tqdm.notebook import tqdm


In [ ]:
print(chamfer_distance(points1_t, points2_t))

chamferDist = ChamferDistance()
print(chamferDist(points1_t, points2_t))

In [ ]:
random.seed = 42

In [ ]:
#path = Path("ModelNet10")
#path = Path('/content/drive/MyDrive/ElementNet/')
path = Path('output/')
#savepath = '/content/drive/MyDrive/ElementNet/'
savepath = 'models/'

## Model

## Test

Analyze results statistically

POINTNET++

In [ ]:
train_transforms = transforms.Compose([
                    Normalize(),
#                    RandomNoise(),
                    ToTensor()
                    ])

In [ ]:
# load data and model
BASE_DIR = os.path.dirname(os.path.abspath('industrial-facility-relationships/'))
BASE_DIR = os.path.join(BASE_DIR, 'pointnet2')
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))
inference = False
if inference:
    path = Path('output/bp_data/')
    ext = ".ply"
#     path = Path('tee_fix/')
#     ext = ".pcd"
else:
    path = Path('output/')
    ext = ".pcd"

cat= 'tee'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_normals = False
cat_targets = {"elbow":14, "tee":19, "pipe":11}

if inference:
    test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms, inference=True)
    targets = cat_targets[cat]
else:
    test_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)
    targets = test_ds.targets

testDataLoader = torch.utils.data.DataLoader(dataset=test_ds, batch_size=32)
test_criterion = nn.MSELoss()

model_name = "pointnet2_cls_ssg"
model_path = Path("pointnet2/log/classification/pointnet2_cls_ssg/")
model = importlib.import_module(model_name)


predictor = model.get_model(targets, normal_channel=use_normals)
if device != "cpu":
    predictor = predictor.cuda()

checkpoint = torch.load(model_path/'checkpoints/best_model.pth')
#checkpoint = torch.load(model_path/'checkpoints/models/best_model_t_chamfer_0005.pth')
predictor.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
blueprint = 'data/sample.ifc'
temp_dir = "output/temp/"
target_dir = "output/tee/test/"

ifcConvert_executable = "scripts/./IfcConvert"
cloudCompare_executable = "cloudcompare.CloudCompare"
sample_size = 2048
threshold = 2

In [ ]:
def model_inference(model, loader, device, calculate_score=False):
    predictor = model.eval()
    predictions_list, pcd_list, id_list = [], [], []
    with torch.no_grad():
        for j, data  in tqdm(enumerate(loader), total=len(loader)):
            points, ids = data['pointcloud'].to(device).float(), data['id'].to(device)
            points = points.transpose(2, 1)
            preds, _ = predictor(points)
            preds, points, ids = preds.to(torch.device('cpu')), points.to(torch.device('cpu')), data['id'].to(torch.device('cpu'))
            for i, pr in enumerate(preds):
                predictions_list.append(pr.numpy())
                pcd_list.append(points[i].numpy())
                id_list.append(ids[i].numpy())

        return (predictions_list, pcd_list, id_list)

In [ ]:
if inference:
    predictions_list, cloud_list, id_list = model_inference(predictor.eval(), testDataLoader, device)

In [ ]:
def test(model, loader, device, criterion):
    losses = []
    predictor = model.eval()
    cloud_list = []
    label_list = []
    output_list = []
    predictions_list = []
    inputs_list = []
    id_list = []
    parameter_id = 0
    tot = 0
    count = 0
    
    for j, data  in tqdm(enumerate(loader), total=len(loader)):
        inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points, target, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        points = points.transpose(2, 1)
        outputs, _ = predictor(points)
        outputs = outputs.to(torch.device('cpu'))
        inputs = points.to(torch.device('cpu'))
        labels = target.to(torch.device('cpu'))
        ids = ids.to(torch.device('cpu'))
        #print(data['pointcloud'].size(), labels.size(), outputs.size())

        for i in range(outputs.size(0)):
            label_list.append(labels[i][parameter_id].item())
            id_list.append(ids[i].item())
            output_list.append(outputs[i][parameter_id].item())
            predictions_list.append(outputs[i].numpy())
            inputs_list.append(labels[i].numpy())
            cloud_list.append(inputs[i].numpy())
            ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
            #print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
            tot += np.absolute(ratio)
            #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
        count += outputs.size(0)
    print(tot/count)

    return predictions_list, inputs_list, label_list, output_list, id_list, cloud_list

In [ ]:
if not inference:
    with torch.no_grad():
        predictions_list, inputs_list, label_list, output_list, id_list, cloud_list = test(predictor.eval(), testDataLoader, device, test_criterion)

    print(len(predictions_list), len(inputs_list))

In [ ]:
# k = 5
# j = 4
# direction = False
# use_direction = True
# square_error = True
# error_calc(predictions_list, inputs_list, k, j, direction, use_direction, square_error)

In [ ]:
if not inference:
    label_list, output_list, id_list = np.array(label_list), np.array(output_list), np.array(id_list)

In [ ]:
# ratio = np.absolute((label_list - output_list)/label_list)
# ratio_ind = ratio.argsort()
# id_list = id_list[ratio_ind]
# print(id_list[-10:-1])

# error_threshold = 0.1
# correct = ratio[np.where(ratio < error_threshold)]
# print(len(ratio), len(correct), len(correct)/len(ratio))


In [ ]:
# cloud_id = 2
# pcd_id = 24229  


#### Visually analyse predictions and Fine tune with ICP, calculate chamfer distances

In [ ]:
# scaling up and down is required for icp calculations
def chamfer_evaluate(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size,
                     threshold, icp_correction = False):

    preds_list, pcd_list = [], []
    error_count = 0

    # get predictions and pcds
    for i in tqdm(range(len(predictions_list))):
    #for i in tqdm(range(50)):
        pcd_id = id_list[i].item()
        pcd, preds = cloud_list[i].transpose(1, 0), copy.deepcopy(predictions_list[i])
        #print(preds, inputs_list[i])

        preds = scale_preds(preds.tolist(), cat)
        #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, predictions_list, path, ext)

        try:
            if  icp_correction:
                # note: preds are updated in place during ICP
                _, _ = icp_finetuning(o3d.utility.Vector3dVector(pcd), pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
                                     ifcConvert_executable, cloudCompare_executable, sample_size, threshold, False)

            preds_list.append(preds)
            pcd_list.append(pcd)

        except Exception as e:
            print("ICP error", pcd_id, e)
            error_count += 1

    # calculate chamfer distances
    cuda = torch.device('cuda')    
    rescaled_preds = [scale_preds(preds, cat, up=0) for preds in preds_list]
    preds_t = torch.tensor(rescaled_preds, requires_grad=True, device=cuda)
    cloud_t = torch.tensor(cloud_list, device=cuda)
    
    chamfer_dists = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False, alpha=3.)
    chamfer_dists = chamfer_dists.detach().cpu().numpy()
    
#     for i, preds in enumerate(tqdm(preds_list)):
#         preds = scale_preds(preds, cat, up=0)
#         chamfer_distance, _ = get_chamfer_dist_single(pcd_list[i], preds, cat)
#         chamfer_dists.append(chamfer_distance)
        

    
    print("error_count", error_count)
    return chamfer_dists

In [ ]:
dists = chamfer_evaluate(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)
# if inference:
#     with open(model_path + 'preds_' + cat + '.pkl', 'wb') as f:
#         pickle.dump([predictions_list, id_list, dists], f)
    
plot_error_graph(dists, "Binned chamfer loss", max_val=300)

In [ ]:
# scaling up and down is required for icp calculations
def _visualise_predictions(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size,
                     threshold, icp_correction = False):

    preds_list, pcd_list = [], []
    viewer_list, ifc_list = [], []
    error_count = 0

    # get predictions and pcds
    #for i in tqdm(range(len(predictions_list))):
    for i in tqdm(range(20)):
        pcd_id = id_list[i].item()
        pcd, preds = cloud_list[i].transpose(1, 0).tolist(), copy.deepcopy(predictions_list[i])
        #print(preds, inputs_list[i])

        preds = scale_preds(preds.tolist(), cat)
        print(preds)
        #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, inputs_list, ext)

#         try:
        if  icp_correction:
            # note: preds are updated in place during ICP
            viewer, ifc = icp_finetuning(o3d.utility.Vector3dVector(pcd), pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
                                 ifcConvert_executable, cloudCompare_executable, sample_size, threshold, True)
        else:
            #print(preds)
            viewer, ifc = visualize_predictions([pcd], cat, [preds], blueprint, visualize=True)


        preds_list.append(preds)
        pcd_list.append(pcd)
        viewer_list.append(viewer)
        ifc_list.append(ifc)

#         except Exception as e:
#             print("ICP error", pcd_id, e)
#             error_count += 1


    print("error_count", error_count)    
    return viewer_list

In [ ]:
viewers = _visualise_predictions(predictions_list, cloud_list, id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

In [ ]:
for v in viewers:
    print(v)

In [ ]:
# icp_correction = False
# viewers, ifcs = [], []
# preds_list, pcd_list = [], []

# for i in range(1,4):
#     pcd_id = id_list[i].item()
#     #print(pcd_id) 
        
#     #print("p", predictions_list[i], "in", inputs_list[i])

# #     #pcd, preds = prepare_visualisation(pcd_id, cat, i, cloud_list, predictions_list, path, ext)
# #     pcd, preds = cloud_list[i].transpose(1, 0), copy.deepcopy(predictions_list[i])
# #     print(pcd.shape)
# #     preds = scale_preds(preds.tolist(), cat)
# # #     pcd2, input_preds = prepare_visualisation(pcd_id, cat, i, cloud_list, inputs_list, path, ext)
# # #     indices_to_replace = [5, 6, 7]
# # #     for i in indices_to_replace:
# # #         preds[i] = input_preds[i]
    
# #     #try:
# #     if not icp_correction:
# #         viewer, ifc = visualize_predictions([pcd], cat, [preds], blueprint, visualize=True)

# #     else:
# #         viewer, ifc = icp_finetuning(pcd, pcd_id, cat, preds, blueprint, temp_dir, target_dir, 
# #                              ifcConvert_executable, cloudCompare_executable, sample_size, threshold)

# #     viewers.append(viewer) 
# #     ifcs.append(ifc)
# #     preds_list.append(preds)
# #     pcd_list.append(pcd)
       
# # #     except Exception as e:
# # #         print("ICP error", pcd_id, e)
        
#     preds_list.append(inputs_list[i])
#     pcd_list.append(cloud_list[i].transpose(1, 0))

# cloud = o3d.geometry.PointCloud()
# chamfer_dists = []
# for i, preds in enumerate(preds_list):
#     pcd = pcd_list[i]
#     chamfer_distance, cl = get_chamfer_dist_single(pcd, preds, cat)
#     chamfer_dists.append(chamfer_distance)
    
# print("chamfer_distance", sum(chamfer_dists)/len(chamfer_dists))
# print("chamfer_distance", chamfer_dists)
# cloud.points = o3d.utility.Vector3dVector(np.array(cl))
# o3d.io.write_point_cloud("generated.pcd", cloud)
# cloud.points = o3d.utility.Vector3dVector(pcd)
# o3d.io.write_point_cloud("pcd.pcd", cloud)


# #tensor based chamfer dist
# preds_tensor = torch.tensor(preds_list).cuda()
# pcd_tensor = torch.transpose(torch.tensor(pcd_list).cuda().float(), 1,2)
# #print(pcd_tensor.shape)
# chamfer_distance = get_chamfer_loss_tensor(preds_tensor, pcd_tensor)
# print("chamfer", chamfer_distance)


# # for i, cl in enumerate(clouds_tensor.detach().cpu().numpy()):
# #     cloud.points = o3d.utility.Vector3dVector(cl)
# #     o3d.io.write_point_cloud("axis_"+str(i)+".pcd", cloud)
# #     cloud.points = pcd_list[i]
# #     o3d.io.write_point_cloud("pcd_"+str(i)+".pcd", cloud)
    
# # cloud = generate_elbow_cloud(preds)
    
# #chamfer_distance = get_chamfer_dist_single(cloud, pcd)
# #print("chamfer_distance", chamfer_distance)




In [ ]:
plot_parameter_errors(inputs_list, predictions_list, cat)

#### BP data Visualisation

In [ ]:
batch_visualise(model_path, blueprint, path, ext, device, ifc=False)

In [ ]:
#merge_clouds(path, 'pipe')

### Misc

In [ ]:
# PCA
#testset_PCA(cloud_list, inputs_list, testDataLoader, cat)

In [ ]:
print(torch. __version__)

In [ ]:
# preds = inputs_list[:512]
# #preds = scale_preds(preds, cat, up=0)
# pcd = cloud_list[1].transpose(1, 0)
# print(preds)

# preds_tensor = torch.tensor(np.array(preds)).cuda().float()
# gen_cloud = generate_tee_cloud_tensor(preds_tensor)
# print(gen_cloud.shape)
# #gen_cloud = generate_pipe_cloud(preds)

# points = gen_cloud.detach().cpu().numpy()
# #points = np.array(gen_cloud)
# cloud = o3d.geometry.PointCloud()
# print(type(points), points.shape)
# cloud.points = o3d.utility.Vector3dVector(points[0])
# #cloud.points = o3d.utility.Vector3dVector(points)
# o3d.io.write_point_cloud("pipe_gen.pcd", cloud)

# cloud.points = o3d.utility.Vector3dVector(pcd)
# o3d.io.write_point_cloud("pipe_input.pcd", cloud)

In [ ]:
# chamfer loss
#v, loss = chamfer_fine_tune(50, 0.0001, predictions_list[0], cloud_list[0], cat, blueprint)

In [ ]:
# multi element Adam with single loss
def chamfer_fine_tune(n_iter, step_size, preds, cloud, cat, blueprint, alpha=1.0, visualise=True):
    # prepare data on gpu and setup optimiser
    cuda = torch.device('cuda')
    preds_copy = copy.deepcopy(preds)
    scaled_original_preds = [scale_preds(pc.tolist(), cat) for pc in preds_copy]
    preds_t = torch.tensor(preds, requires_grad=True, device=cuda)
    cloud_t = torch.tensor(cloud, device=cuda)
    optimiser = torch.optim.Adam([preds_t], lr=step_size*3, betas=(0.7, 0.9) ,foreach=True)

    # check initial loss
    chamfer_loss = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False, alpha=alpha)
    print("intial loss", chamfer_loss)   

    # iterative refinement with adam
    for i in tqdm(range (n_iter)):
        optimiser.zero_grad()
        chamfer_loss = get_chamfer_loss_tensor(preds_t, cloud_t, cat, alpha=alpha)
        chamfer_loss.backward()
        optimiser.step()

        print(i, "loss", chamfer_loss.detach().cpu().numpy())#, "preds", preds_t)
        
    # check final loss
    chamfer_loss = get_chamfer_loss_tensor(preds_t, cloud_t, cat, reduce=False, alpha=alpha)
    print("final loss", chamfer_loss)
    modified_preds = preds_t.detach().cpu().numpy()
    
    # visualise
    if visualise:
        error_count = 0
        scaled_preds = [scale_preds(p.tolist(), cat) for p in modified_preds]
        visualisers = []
        
        for i, p in enumerate(scaled_preds):
            try:
                v_orignal, _ = visualize_predictions([cloud[i].transpose(1,0).tolist()], cat, [scaled_original_preds[i]], blueprint, visualize=True)
                v_modified, _ = visualize_predictions([None, None, cloud[i].transpose(1,0).tolist()], cat, [scaled_preds[i]], blueprint, visualize=True)
                visualisers.append(v_orignal)
                visualisers.append(v_modified)
            except:
                error_count += 1
    
#         return v_orignal,v_modified, modified_preds
        print("errors ", error_count)
        return visualisers, modified_preds
    else:
        return modified_preds

In [ ]:
print(len(predictions_list))

In [ ]:
#limit = 20
limit = len(predictions_list)

#v, modified_preds = chamfer_fine_tune(50, 0.01, predictions_list[:limit], cloud_list[:limit], cat, blueprint, alpha=3., visualise=True)
modified_preds = chamfer_fine_tune(50, 0.01, predictions_list[:limit], cloud_list[:limit],
                                   cat, blueprint, alpha=3., visualise=False)


In [ ]:
#print(v)

In [ ]:
plot_parameter_errors(inputs_list, modified_preds, cat)

In [ ]:
dists = chamfer_evaluate(modified_preds, cloud_list[:limit], id_list, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

if inference:
    with open(model_path/('preds_basin_finetuned_' + cat + '.pkl'), 'wb') as f:
        pickle.dump([modified_preds, id_list, dists], f)

plot_error_graph(dists, "Binned chamfer loss", max_val=300)

In [ ]:

"""
warn: hacky fix
the termination condition of most methods like newton-CG is based on no. of iterations.
But there can be an uncapped number of gradient computations inside a single iteration.
This can make the optimisation stuck. 
This method counts function evaluations within the loss function call itself using a 
global variable, and terminates when a threshold of gradient evaluations are reached.
the callback is unsuitable for this since its only called after each iteration.
"""

# simulated annealing optimisation
from scipy.optimize import basinhopping, minimize
def c_loss(preds):
    #print(i)
    global eval_count
    global results
    eval_count += 1
    #print(eval_count)
    
    # hacky method to stop optimizer iterations based on grad evaluations
    if eval_count > 50:
        results.append(preds)
        raise StopIteration("gradient evals exceeded")
    
    cloud = cloud_list[i]
    #cloud = cloud_list2[4]
    cuda = torch.device('cuda')
    #cloud = cloud.transpose(1,0)
    preds_copy = copy.deepcopy(preds)
    scaled_original_preds = scale_preds(preds_copy.tolist(), cat)
    preds_t = torch.tensor([preds], requires_grad=True, device=cuda, dtype=torch.float)
    #print(preds_t.grad)
    cloud_t = torch.tensor([cloud], device=cuda)
    chamfer_loss = get_chamfer_loss_tensor(preds_t, cloud_t, cat, alpha=3., reduce=True)
    chamfer_loss.backward()
    #print("ch", chamfer_loss.item())
    #print(preds_t.grad.detach().cpu().numpy()[0])
    return chamfer_loss.item(), preds_t.grad.detach().cpu().numpy()[0]
    #return chamfer_loss.item()
   
    
def basinhopping_single(n_iter, step_size, preds, cloud, cat, blueprint):
    minimizer_kwargs = {"method": "CG", "jac":True}
    #minimizer_kwargs = {"method": "BFGS"}
    ret = basinhopping(c_loss, preds, stepsize=step_size, minimizer_kwargs=minimizer_kwargs, 
                       niter=n_iter)
    print("global minimum: ", ret)
    return ret
    
    # visualise
    # modified_preds = preds_t.detach().cpu().numpy()[0].tolist()
    # scaled_preds = scale_preds(modified_preds, cat)
    # v_orignal, _ = visualize_predictions([cloud.transpose(1,0).tolist()], cat, [scaled_original_preds], blueprint, visualize=True)
    # v_modified, _ = visualize_predictions([cloud.transpose(1,0).tolist()], cat, [scaled_preds], blueprint, visualize=True)
    
    #return [v_orignal,v_modified], chamfer_loss.item()
    

In [ ]:
def callback_fn(intermediate_result):
    #print(intermediate_result["x"], "ch", intermediate_result["fun"])
    print(intermediate_result)
    raise StopIteration("gradient evals exceeded")
    
    
def minimize_single(n_iter, preds, step_size):
    ret = minimize(c_loss, preds, method = "Newton-CG", jac = True, 
                   options = {"maxiter":n_iter, "eps":step_size, "disp":False})
    print("min reached")
    results.append(ret["x"])
#     ret = minimize(c_loss, preds, method = "TNC", jac = True, 
#                    options = {"maxfun":n_iter, "eps":step_size})
    return ret
    

In [ ]:
print(len(predictions_list))
evaluation_limit = 1000

In [ ]:
#c_loss(2, 0.01, predictions_list[0], cloud_list[0], cat, blueprint)
results = []
for i in tqdm(range(evaluation_limit)):
    eval_count = 0
    try:
        minimize_single(25, predictions_list[i], 0.0001)
    except StopIteration:
        pass

In [ ]:
print(results)

In [ ]:
modified_preds = chamfer_fine_tune(10, 0.01, predictions_list[:evaluation_limit], 
                                   cloud_list[:evaluation_limit], cat, blueprint, 
                                   alpha=3., visualise=False)

In [ ]:
# modified_preds = []
# for i in range(evaluation_limit):
#     modified_pred = chamfer_fine_tune(10, 0.01, [predictions_list[i]], 
#                                    [cloud_list[i]], cat, blueprint, 
#                                    alpha=3., visualise=False)
#     modified_preds.append(modified_pred[0])
# modified_preds = np.array(modified_preds)

In [ ]:
#modified_preds

In [ ]:
dists = chamfer_evaluate(predictions_list[:evaluation_limit], cloud_list[:evaluation_limit], id_list, cat, 
                         blueprint,  ifcConvert_executable, cloudCompare_executable, 
                         temp_dir, target_dir, sample_size, threshold, icp_correction = False)

plot_error_graph(dists, "Binned chamfer loss", max_val=300)
print(dists)

In [ ]:
dists = chamfer_evaluate(modified_preds, cloud_list[:evaluation_limit], id_list, cat, 
                         blueprint,  ifcConvert_executable, cloudCompare_executable, 
                         temp_dir, target_dir, sample_size, threshold, icp_correction = False)

plot_error_graph(dists, "Binned chamfer loss", max_val=300)
print(dists)

In [ ]:
dists = chamfer_evaluate(results, cloud_list[:evaluation_limit], id_list, cat, 
                         blueprint,  ifcConvert_executable, cloudCompare_executable, 
                         temp_dir, target_dir, sample_size, threshold, icp_correction = False)

plot_error_graph(dists, "Binned chamfer loss", max_val=300)
print(dists)

In [ ]:
plot_parameter_errors(inputs_list[:evaluation_limit], results, cat)

In [ ]:
# second round of fine tuning
print(sum(dists)/len(dists))
threshold = 0.02
start_point = int(threshold*len(dists))
print(start_point)

indices = dists.argsort()
#print(dists[indices][-1*start_point:])
mod_pred2 = modified_preds[indices][-1*start_point:]
cloud_list2 = np.array(cloud_list)[indices][-1*start_point:]
id_list2 = np.array(id_list)[indices][-1*start_point:]
print(len(mod_pred2), max(dists))
# modified_preds2 = chamfer_fine_tune(50, 0.001, mod_pred2, cloud_list2, cat, blueprint, alpha=3, visualise=False)
# dists2 = chamfer_evaluate(modified_preds2, cloud_list2, id_list2, cat, blueprint,  ifcConvert_executable,
#                      cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

In [ ]:
dists[indices][-1*start_point:]

In [ ]:
ret = basinhopping_single(1, 0.01, mod_pred2[4], cloud_list2[7], cat, blueprint)


In [ ]:
modified_preds = []
funs = []
for i in range(len(mod_pred2)):
    ret = basinhopping_single(1, 0.01, mod_pred2[i], cloud_list2[i], cat, blueprint)
    modified_preds.append(ret.x)
    funs.append(ret.fun)
print(len(modified_preds))

In [ ]:
file = open('basinhopped_res.pkl', 'wb')

# dump information to that file
pickle.dump([modified_preds, funs], file)

In [ ]:
dists2 = chamfer_evaluate(modified_preds, cloud_list2, id_list2, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)
dists2

In [ ]:
preds = torch.tensor([predictions_list[1]]).cuda()
print(predictions_list[1])
pcd = generate_tee_cloud_tensor(preds)
#tee = generate_tee_cloud(predictions_list[0])
tee = pcd[0].cpu().numpy()
tee = o3d.utility.Vector3dVector(tee)
tee_cloud = o3d.geometry.PointCloud()
tee_cloud.points = tee
o3d.io.write_point_cloud("tee_cl.pcd", tee_cloud)


In [ ]:
original = cloud_list[2]
points= o3d.utility.Vector3dVector(original.transpose(1,0))
tee_cloud.points = points
o3d.io.write_point_cloud("tee_cl_inp.pcd", tee_cloud)

In [ ]:
# undo normalisation opf the bp tee dataset created for inference, only for comparison with the inferred tee results
tee_path = 'tee_fix/tee/test/'
metadata_file = open("tee_fix/tee/metadata.json", 'r')
metadata = json.load(metadata_file)
output_path = 'tee_fix/tee/unnormalised/'

files = os.listdir(tee_path)
new_points = []
for f in tqdm(files):
    cloud_data = metadata[f.split(".")[0]]
    points = np.array(o3d.io.read_point_cloud(tee_path + f).points)
    print("a", points[0])
    print(cloud_data["norm_factor"], cloud_data["mean"])
    points *= cloud_data["norm_factor"]
    print("b", points[0])

    for i, pnt in enumerate(points):
        pnt += cloud_data["mean"]
    print("c", points[10])
    new_points.append(points)
        
new_points = o3d.utility.Vector3dVector(np.concatenate(new_points))
new_cloud = o3d.geometry.PointCloud()
new_cloud.points = new_points
o3d.io.write_point_cloud(output_path+"tee_bp_unnormalised.pcd", new_cloud)

## Mesh deformation

In [ ]:
from src.finetune import chamfer_fine_tune_single
modified_preds = []
for i in range(len(mod_pred2)):
    ft = chamfer_fine_tune(20, 0.01, [mod_pred2[i]], [cloud_list2[i]], cat, blueprint, alpha=3., visualise=False)
    modified_preds.append(ft[0])
print(len(modified_preds))

In [ ]:
modified_preds2 = chamfer_fine_tune(50, 0.02, modified_preds, cloud_list, cat, blueprint, alpha=3, visualise=False)
dists2 = chamfer_evaluate(modified_preds2, cloud_list2, id_list2, cat, blueprint,  ifcConvert_executable,
                     cloudCompare_executable, temp_dir, target_dir, sample_size, threshold, icp_correction = False)

In [ ]:
print(ret.x, ret.fun)